Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-08-14 22:24:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2,5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2,55M  1,25MB/s    in 2,0s    

2019-08-14 22:24:25 (1,25 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
from tensorflow.keras.callbacks import Callback

class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('acc')>DESIRED_ACCURACY):
            print('\n reached 99%')
            self.model.stop_training = True

callbacks = myCallback()

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [24]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
    Conv2D(input_shape = (150,150,3),filters = 16, kernel_size = 5, activation='relu'),
    MaxPool2D(pool_size = 2),
    Conv2D(filters = 32, kernel_size = 5, activation='relu'),
    MaxPool2D(pool_size = 2),
    Conv2D(filters = 64, kernel_size = 5, activation='relu'),
    MaxPool2D(pool_size = 2),
    Flatten(),
    Dense(units = 128, activation=tf.nn.relu),
    Dense(units = 2, activation=tf.nn.sigmoid)    
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 146, 146, 16)      1216      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 69, 69, 32)        12832     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 34, 34, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 64)        51264     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 14400)            

In [16]:
from tensorflow.keras.optimizers import RMSprop

opt = RMSprop(lr=0.001)

model.compile(optimizer=opt, loss = "binary_crossentropy", metrics=["acc"])

In [21]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    directory='/tmp/h-or-s',
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [22]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15


ValueError: A target array with shape (16, 1) was passed for an output of shape (None, 2) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.